# 第五章 Pandas入门

## 数据结构

### 创建Series Dataframe 对象

传入嵌套字典

In [17]:
from pandas import Series,DataFrame
import pandas as np

In [11]:
pop={'Nevada':{2001:2.4,2002:2.9},'Ohio':{2000:1.5,2002:3.6}}
f1=DataFrame(pop)
f1

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,NaN
2002,2.9,3.6


### index的方法和属性<br>
方法|说明
:--|:--
`append` |连接两个 `index` ，产生一个**新的 Index**                                                   
`delete` |删除索引 `i` 处的元素，并得到**新的 Index**
`is_monotonic` |是否单调递增
`is_unique` |`index` 中没有重复项时，返回 `True`
`unique` |计算 index 中唯一值的数组